<a href="https://colab.research.google.com/github/brokentwigs/Colab-Analysis/blob/main/PM4PY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Package Installation

In [ ]:
pip install pm4py

## All Imports

In [ ]:
import pm4py

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import os

In [ ]:
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.util import func as functools
from pm4py.algo.discovery.alpha import algorithm as alpha_miner

In [ ]:
from pm4py.algo.conformance.tokenreplay import algorithm as token_replay
from pm4py.algo.discovery.inductive import algorithm as inductive_miner

In [ ]:
import time

In [ ]:
import pandas as pd

import numpy as np

import sklearn

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report,roc_auc_score,confusion_matrix,accuracy_score,f1_score,roc_curve

from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential

from keras.callbacks import ReduceLROnPlateau, EarlyStopping

from keras.layers import Activation, Dense, Dropout, Embedding, LSTM

import re

from IPython.display import display

import os

import string

import time

import random

import matplotlib.pyplot as plt

from tensorflow.keras.datasets import imdb

## Temporal Splitting

In [ ]:
testing_length = len(df_bpi['case'].unique())//10
training_length = len(df_bpi['case'].unique()) - (len(df_bpi['case'].unique())//10)
training_cases = df_bpi['case'].unique()[0:training_length]
testing_cases = df_bpi['case'].unique()[training_length:]
df_bpi_train = df_bpi[df_bpi['case'].isin(training_cases)]
df_bpi_train
df_bpi_test = df_bpi[df_bpi['case'].isin(testing_cases)]

,"""case""","""event""","""startTime""","""completeTime""","""AMOUNT_REQ""","""REG_DATE""","""org:resource"""
0,"""173688""","""A_SUBMITTED""","""2011/10/01 00:38:44.546""","""2011/10/01 00:38:44.546""","""20000""","""2011/10/01 00:38:44.546""","""112"""
1,"""173688""","""A_PARTLYSUBMITTED""","""2011/10/01 00:38:44.880""","""2011/10/01 00:38:44.880""","""20000""","""2011/10/01 00:38:44.546""","""112"""
2,"""173688""","""A_PREACCEPTED""","""2011/10/01 00:39:37.906""","""2011/10/01 00:39:37.906""","""20000""","""2011/10/01 00:38:44.546""","""112"""
3,"""173688""","""A_PREACCEPTED""","""2011/10/01 00:39:37.906""","""2011/10/01 00:39:37.906""","""20000""","""2011/10/01 00:38:44.546""","""112"""
4,"""173688""","""W_Completeren aanvraag""","""2011/10/01 11:36:46.437""","""2011/10/01 11:45:13.917""","""20000""","""2011/10/01 00:38:44.546""","""112"""
...,...,...,...,...,...,...,...
139409,"""203179""","""W_Nabellen offertes""","""2012/01/25 09:44:54.430""","""2012/01/25 09:45:26.288""","""20000""","""2012/01/23 11:36:27.591""","""10982"""
139410,"""203179""","""W_Nabellen offertes""","""2012/02/01 09:37:47.809""","""2012/02/01 09:40:39.661""","""20000""","""2012/01/23 11:36:27.591""","""10861"""
139411,"""203179""","""W_Nabellen offertes""","""2012/02/02 19:00:45.678""","""2012/02/02 19:02:22.452""","""20000""","""2012/01/23 11:36:27.591""","""11180"""
139412,"""203179""","""O_CANCELLED""","""2012/02/02 19:02:20.159""","""2012/02/02 19:02:20.159""","""20000""","""2012/01/23 11:36:27.591""","""11180"""


## Playing Around with Pm4py

In [ ]:
log_csv = pd.read_csv('RoadFines.csv', sep=',')
#log_csv.rename(columns={'case': 'case:clientID'}, inplace=True)
parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'case'}
event_log = log_converter.apply(log_csv, parameters=parameters, variant=log_converter.Variants.TO_EVENT_LOG)

ParserError: ignored

In [ ]:
trace_list = list(filter(lambda i:len(i)<5,event_log))
trace_list

In [ ]:
def import_csv(csv_path):
  event_log_csv = pd.read_csv(csv_path,sep = ',')
  event_log_csv = pm4py.format_dataframe(event_log_csv,case_id = 'case', activity_key = 'event', timestamp_key = 'startTime',timest_format='%Y/%m/%d %H:%M:%S.%f')
  start_activities = pm4py.get_start_activities(event_log_csv)
  end_activities = pm4py.get_end_activities(event_log_csv)
  print('Start_activities:{}\n End_activites:{}'.format(start_activities, end_activities))

## BPMN Model Creation

In [ ]:
import_csv('RoadFines.csv')

In [ ]:
df = pm4py.format_dataframe(pd.read_csv('RoadFines.csv',sep = ','),case_id = 'case', activity_key = 'event', timestamp_key = 'startTime')

In [ ]:
#The estimated runtime of the process for the given dataset was: 32 seconds

bpmn_model = pm4py.discover_bpmn_inductive(df)

#print("The Average Time taken for constructing the bpmn model on the given database:",sum(l)/len(l) ,'seconds')
pm4py.view_bpmn(bpmn_model)

## BPMN Conformance Checking

### Token Based Replay

In [ ]:
log_xes = xes_importer.apply(os.path.join("RoadFines.xes"))
net, initial_marking, final_marking = alpha_miner.apply(log_xes)
replayed_traces = token_replay.apply(log_xes, net, initial_marking, final_marking)

In [ ]:
replayed_traces

### Alignments

In [ ]:
log_xes = xes_importer.apply(os.path.join("RoadFines.xes"))
net_al, initial_marking_al, final_marking_al = inductive_miner.apply(log_xes)

from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
aligned_traces = alignments.apply_log(log_xes, net_al, initial_marking_al, final_marking_al)

In [ ]:
net_al
from pm4py.visualization.petri_net import visualizer as pn_visualizer
gviz = pn_visualizer.apply(net_al, initial_marking_al, final_marking_al)
pn_visualizer.view(gviz)

# LSTM Research

In [ ]:
#fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from keras.callbacks import EarlyStopping
from keras.layers import ConvLSTM2D
from sklearn.preprocessing import StandardScaler

df_lstm = pd.read_csv('RoadFines.csv')

In [ ]:
train_dates = pd.to_datetime(df_lstm['startTime'])

In [ ]:
cols = list(df_lstm)[0:2]+list(df_lstm)[3:]

In [ ]:
df_for_training =  df_lstm[cols]
df_for_plot = df_for_training
df_for_plot.plot.line()